<a href="https://colab.research.google.com/github/NeginShams/Quran_QA/blob/main/xlm_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install datasets sentencepiece

In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.5 MB/s eta 0:00:00


In [4]:
pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-07q406lm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-07q406lm
  Resolved https://github.com/huggingface/accelerate to commit 5f21cde3c74fa511c4bec55cac5fe410effbe182
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=227410 sha256=780ea989db178dfda51311349220a9ddf80979879e2e6b22becda9563d4e6070
  Stored in directory: /tmp/pip-ephem-wheel-cache-y37dq8av/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.19.0
    Uninstalling accelerate-0.19.0:
      Succe

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -e.

In [5]:
# pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [ ]:
# from transformers import AutoModelForMaskedLM, AutoTokenizer

# # v3.0
# model_name_or_path = "xlm-roberta-base"

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# model = AutoModelForMaskedLM.from_pretrained(model_name_or_path)

In [7]:
from transformers import XLMRobertaForMaskedLM, XLMRobertaTokenizer

# v3.0
model_name_or_path = "xlm-roberta-base"

tokenizer = XLMRobertaTokenizer.from_pretrained(model_name_or_path)

model = XLMRobertaForMaskedLM.from_pretrained(model_name_or_path)

In [8]:
quranpath = '/content/drive/MyDrive/quran.txt'
with open(quranpath , 'r' , encoding= 'utf-8') as f:
    lines = f.readlines()

for i in range(len(lines)):
  lines[i] = lines[i][:-1]

In [9]:
import numpy as np

verse_lengths = [len(x.split()) for x in lines]
maxlength =  max(verse_lengths)
maxlengthid =  np.argmax([verse_lengths])
verse_lengths = sorted([(i , x) for i , x in enumerate(verse_lengths)] ,key =lambda x : x[1] )
maxlength

301

In [10]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': [quranpath], 'test':quranpath})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-a2850252c4ff5c38/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6236
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6236
    })
})

In [11]:
def tokenize_pad_and_truncate(texts):
    return tokenizer(texts["text"], padding="max_length")

tokenized_datasets = dataset.map(tokenize_pad_and_truncate, batched=True)

Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

In [12]:
def tokenize_pad_and_truncate(texts):
    return tokenizer(texts["text"], padding="max_length")

tokenized_datasets = dataset.map(tokenize_pad_and_truncate, batched=True)

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.1)

In [14]:
import transformers
print(transformers.__version__)

4.29.2


In [15]:
from transformers import  TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Quran_QA4/quranBERT_moreepochs",
    save_strategy='epoch',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs= 1,
    weight_decay=0.01,
)

In [16]:
from random import randint
testset =tokenized_datasets['test'].select([randint(0 , 6000) for _ in range(200)])
testset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 200
})

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'] ,
    eval_dataset = testset,
    data_collator=data_collator,
    tokenizer= tokenizer
)

In [ ]:
len(tokenized_datasets['train'])

6236

In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.823900,nan


TrainOutput(global_step=3118, training_loss=1.9479721049760839, metrics={'train_runtime': 1933.9241, 'train_samples_per_second': 3.225, 'train_steps_per_second': 1.612, 'total_flos': 1645549789224960.0, 'train_loss': 1.9479721049760839, 'epoch': 1.0})

In [19]:
model.save_pretrained("/content/drive/MyDrive/Quran_QA4/quranBERT/8epochs")
tokenizer.save_pretrained("/content/drive/MyDrive/Quran_QA4/quranBERT/8epochs")

('/content/drive/MyDrive/Quran_QA4/quranBERT/8epochs/tokenizer_config.json',
 '/content/drive/MyDrive/Quran_QA4/quranBERT/8epochs/special_tokens_map.json',
 '/content/drive/MyDrive/Quran_QA4/quranBERT/8epochs/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Quran_QA4/quranBERT/8epochs/added_tokens.json')

In [21]:
sample = tokenizer(["ما را به راه <mask> هدایت کن"] , return_tensors='pt').to("cuda")
result =model(**sample)

ids =np.argmax(result[0].detach().cpu().numpy(), axis= 2)
print(ids)
tokenizer.decode(ids[0])

[[     0    877    406    178   7380  43336 147343  10302      2]]


'<s> ما را به راه راست هدایت کن</s>'

In [22]:
sample = tokenizer(["به‌نام خداوند بخشنده و <mask>"] , return_tensors='pt').to("cuda")
result =model(**sample)

ids =np.argmax(result[0].detach().cpu().numpy(), axis= 2)
print(ids)
tokenizer.decode(ids[0])

[[     0    178   2618  54335   6771  12157     65 109932      2]]


'<s> به نام خداوند بخشنده و مهربان</s>'